## Request Index page for top 500 alexa websites
- Analysis per CDN provider
    - time to first byte
    - time to DNS resolution
    - time to TCP connection
    - time to SSL negotiation
    - receive time

In [1]:
from __future__ import division
%pylab inline
import pandas as pd
import numpy as np
import subprocess
import os, sys, re
from collections import defaultdict

Populating the interactive namespace from numpy and matplotlib


In [2]:
#df_sites = pd.read_csv('data/top-1m-new.csv', nrows=500, header = None, names = ['rank', 'site'])
#df_asn_cdn = pd.read_pickle('output/df_asn_cdn.pkl')

df_sites = pd.read_pickle('output/df_sites.pkl')

df_sites.head()

,rank,site,IP,ASN
0,1,google.com,172.217.160.238,15169
1,2,youtube.com,172.217.31.14,15169
2,3,facebook.com,157.240.25.35,32934
3,4,baidu.com,123.125.115.110,4808
4,5,wikipedia.org,103.102.166.224,14907


### CURL Command Example with Output
curl -v --trace-time -o /dev/null --connect-timeout 3.0 -w "\n\nContentType: %{content_type}\nResponseCode: %{response_code}\nURLEffective: %{url_effective}\n\nSizeDownload: %{size_download};\tSizeRquest: %{size_request}\nSizeUpload: %{size_upload};\tSizeHeader: %{size_header}\n\nTimeNameLookup: %{time_namelookup}\nTimeConnect: %{time_connect}\nTimeAppConnect: %{time_appconnect}\nTimePreTransfer: %{time_pretransfer}\nTimeRedirect: %{time_redirect}\nTimeStartTransfer: %{time_starttransfer}\nTimeTotal: %{time_total}\n" -s https://www.google.co.in


12:24:41.186600 * Rebuilt URL to: https://www.google.co.in/  
12:24:41.191017 *   Trying 172.217.167.3...  
12:24:41.191061 * TCP_NODELAY set  
12:24:41.218682 * Connected to www.google.co.in (172.217.167.3) port 443 (#0)  
12:24:41.218794 * ALPN, offering h2  
12:24:41.218825 * ALPN, offering http/1.1  
12:24:41.218890 * Cipher selection: ALL:!EXPORT:!EXPORT40:!EXPORT56:!aNULL:!LOW:!RC4:@STRENGTH  
12:24:41.224963 * successfully set certificate verify locations:  
12:24:41.224997 *   CAfile: /etc/ssl/cert.pem  
  CApath: none  
12:24:41.225187 * TLSv1.2 (OUT), TLS handshake, Client hello (1):  
12:24:41.225219 } [222 bytes data]  
12:24:41.302091 * TLSv1.2 (IN), TLS handshake, Server hello (2):  
12:24:41.302182 { [96 bytes data]  
12:24:41.302706 * TLSv1.2 (IN), TLS handshake, Certificate (11):  
12:24:41.302769 { [3128 bytes data]  
12:24:41.309234 * TLSv1.2 (IN), TLS handshake, Server key exchange (12):  
12:24:41.309275 { [116 bytes data]  
12:24:41.312864 * TLSv1.2 (IN), TLS handshake, Server finished (14):  
12:24:41.312896 { [4 bytes data]  
12:24:41.313614 * TLSv1.2 (OUT), TLS handshake, Client key exchange (16):  
12:24:41.313644 } [37 bytes data]  
12:24:41.313695 * TLSv1.2 (OUT), TLS change cipher, Client hello (1):  
12:24:41.313723 } [1 bytes data]  
12:24:41.313895 * TLSv1.2 (OUT), TLS handshake, Finished (20):  
12:24:41.313925 } [16 bytes data]  
12:24:41.318727 * TLSv1.2 (IN), TLS change cipher, Client hello (1):  
12:24:41.318799 { [1 bytes data]  
12:24:41.319048 * TLSv1.2 (IN), TLS handshake, Finished (20):  
12:24:41.319106 { [16 bytes data]  
12:24:41.319209 * SSL connection using TLSv1.2 / ECDHE-ECDSA-CHACHA20-POLY1305  
12:24:41.319265 * ALPN, server accepted to use h2  
12:24:41.319324 * Server certificate:  
12:24:41.319405 *  subject: C=US; ST=California; L=Mountain View; O=Google LLC; CN=*.google.com  
12:24:41.319470 *  start date: Nov  7 08:59:00 2018 GMT  
12:24:41.319525 *  expire date: Jan 30 08:59:00 2019 GMT  
12:24:41.319705 *  subjectAltName: host "www.google.co.in" matched cert's "*.google.co.in"  
12:24:41.319858 *  issuer: C=US; O=Google Trust Services; CN=Google Internet Authority G3  
12:24:41.319915 *  SSL certificate verify ok.  
12:24:41.319996 * Using HTTP2, server supports multi-use  
12:24:41.320050 * Connection state changed (HTTP/2 confirmed)  
12:24:41.320106 * Copying HTTP/2 data in stream buffer to connection buffer after upgrade: len=0  
12:24:41.320264 * Using Stream ID: 1 (easy handle 0x7fb146006600)  
12:24:41.320351 > GET / HTTP/2  
12:24:41.320351 > Host: www.google.co.in  
12:24:41.320351 > User-Agent: curl/7.54.0  
12:24:41.320351 > Accept: */*  
12:24:41.320351 >  
12:24:41.324419 * Connection state changed (MAX_CONCURRENT_STREAMS updated)!  
12:24:41.425512 < HTTP/2 200  
12:24:41.425586 < date: Tue, 11 Dec 2018 06:54:41 GMT  
12:24:41.425637 < expires: -1  
12:24:41.425687 < cache-control: private, max-age=0  
12:24:41.425741 < content-type: text/html; charset=ISO-8859-1  
12:24:41.425793 < p3p: CP="This is not a P3P policy! See g.co/p3phelp for more info."  
12:24:41.425845 < server: gws  
12:24:41.425897 < x-xss-protection: 1; mode=block  
12:24:41.425949 < x-frame-options: SAMEORIGIN  
12:24:41.426026 < set-cookie: 1P_JAR=2018-12-11-06; expires=Thu, 10-Jan-2019 06:54:41 GMT; path=/; domain=.google.co.in  
12:24:41.426180 < set-cookie: NID=150=pPHcNvIyWCtqTFQg8ywE9W6ZJEq7IgUcIdKsn5Od5SUcrW19sHBUblridrGt7zMn9IyRgUhJqvrJ0Hw5B9kkW51nJMsBQ9yTzwCtUYZUnGKq5E7sD5X7zNKmNXV7mmljGbb9CIqk1XigbExIVNozjA9uO_BJoPaO5gW93g_Z91I; expires=Wed, 12-Jun-2019 06:54:41 GMT; path=/; domain=.google.co.in; HttpOnly  
12:24:41.426250 < alt-svc: quic=":443"; ma=2592000; v="44,43,39,35"  
12:24:41.426303 < accept-ranges: none  
12:24:41.426377 < vary: Accept-Encoding  
12:24:41.426457 <  
12:24:41.426640 { [1388 bytes data]  
12:24:41.428591 * Connection #0 to host www.google.co.in left intact  



ContentType: text/html; charset=ISO-8859-1  
ResponseCode: 200  
URLEffective: https://www.google.co.in/  



SizeDownload: 13548;	SizeRquest: 78  
SizeUpload: 0;	SizeHeader: 759  



TimeNameLookup: 0.004405  
TimeConnect: 0.032078  
TimeAppConnect: 0.133374  
TimePreTransfer: 0.133973  
TimeRedirect: 0.000000  
TimeStartTransfer: 0.240047  
TimeTotal: 0.241995  

### Regarding curl timings from https://ec.haxx.se/usingcurl-verbose.html:  

%{time_appconnect} shows the time, in seconds, it took from the start until the SSL/SSH/etc connect/handshake to the remote host was completed.  

%{time_connect} shows the time, in seconds, it took from the start until the TCP connect to the remote host (or proxy) was completed.  

%{time_namelookup} shows the time, in seconds, it took from the start until the name resolving was completed.  

%{time_pretransfer} shows the time, in seconds, it took from the start until the file transfer was just about to begin. This includes all pre-transfer commands and negotiations that are specific to the particular protocol(s) involved.  

%{time_redirect} shows the time, in seconds, it took for all redirection steps including name lookup, connect, pre-transfer and transfer before the final transaction was started. time_redirect shows the complete execution time for multiple redirections.  

%{time_starttransfer} shows the time, in seconds, it took from the start until the first byte was just about to be transferred. This includes time_pretransfer and also the time the server needed to calculate the result.  

%{time_total} shows the total time, in seconds, that the full operation lasted. The time will be displayed with millisecond resolution.  


## Analyze curl timings data

### Calculate Timings

T0 = timer.start()    
Tf = timer.stop()  


if not using HTTPS/SSL: time_appconnect returned is 0, so set time_appconnect = time_connect

t_dns = time for DNS resolution (no redirects) = time_namelookup - time_redirect  
t_tcp = time for TCP connection (SYN/SYNACK) = time_connect - time_namelookup  
t_ssl = time for SSL handshake (only if https) = time_appconnect - time_connect  
t_fbyte = time_starttransfer  
t_wait = time between issuing GET request and first byte received (server calculation time) = time_starttransfer - time_pretransfer  
t_rx = time to receive data from first to last byte = time_total - time_starttransfer  


t_calc = waiting time between SSL and GET request = time_pretransfer - time_appconnect  
t_start = waiting time between issuing curl command and issuing first byte of data for DNS lookup  
t_stop = waiting time between receiving last byte and ending the curl command  

In [87]:
CURLdata = pd.read_json('output/curl-timing-data-reorder-count100-sites500.json')
CURLdata3 = pd.read_json('output/curl-timing-data-reorder-count1-sites5.json')
CURLdata4 = pd.read_json("output/curl-timing-bad_sites-count20-sites50.json")

# for testing
df = CURLdata4.copy()

### Calculate timings for valid requests

In [74]:
def get_dns_res_time(d):
    return d['time_namelookup'] - d['time_redirect']

def get_tcp_conn_time(d):
    return d['time_connect'] - d['time_namelookup']

#def get_ssl_handshake_time(t_aconn, t_conn):
#    if t_aconn == 0:    # no ssl was used
#        return 0
#    return t_aconn - t_conn

def get_ssl_handshake_time(d):
    t_ssl = d['time_appconnect'] - d['time_connect']
    if t_ssl <= 0:    # no ssl was used
        return 0
    return t_ssl

def get_time_to_first_byte(d):
    return d['time_starttransfer'] #- d['time_pretransfer']

def get_time_to_wait(d):
    return d['time_starttransfer'] - d['time_pretransfer']

def get_data_receive_time(d):
    return d['time_total'] - d['time_starttransfer']


def calculate_timings(df):
    
    #df['t_dns'] = df.apply(lambda r: get_dns_res_time(r), axis=1)
    df['t_dns'] = df.apply(get_dns_res_time, axis=1)
    
    #df['t_tcp'] = df.apply(lambda r: get_tcp_conn_time(r), axis=1)
    #df['t_tcp'] = df.apply(get_tcp_conn_time, axis=1)
    
    df['t_ssl'] = df.apply(lambda r: get_ssl_handshake_time(r), axis=1)
    df['ssl_flag'] = df['time_appconnect'] != 0
    # needs to be done by row for checking if ssl was used
    
    #df['t_fbyte'] = df.apply(lambda r: get_time_to_first_byte(r), axis=1)
    df['t_fbyte'] = df.apply(get_time_to_first_byte, axis=1)
    
    df['t_wait'] = df.apply(get_time_to_wait, axis=1)
    
    #df['t_rx'] = df.apply(lambda r: get_data_receive_time(r), axis=1)
    df['t_rx'] = df.apply(get_data_receive_time, axis=1)
    
    return

In [72]:
calculate_timings(df)

In [73]:
df['pass'] = df['response_code']==200
df.head()

,response_code,size_download,size_header,size_request,size_upload,url_effective,time_namelookup,time_connect,time_appconnect,time_pretransfer,...,time_starttransfer,time_total,rank,t_dns,ssl_flag,t_ssl,t_fbyte,t_rx,pass,t_wait
0,200,13794,755,76,0,https://www.google.com/,1.538197,1.542425,1.639461,1.639623,...,1.757352,1.758427,1,1.538197,True,0.097036,1.757352,0.001075,True,0.117729
1,200,417194,976,77,0,https://www.youtube.com/,0.015011,0.018685,0.126136,0.126297,...,0.237195,1.081214,2,0.015011,True,0.107451,0.237195,0.844019,True,0.110898
2,200,673635,788,78,0,https://www.facebook.com/,0.029400,0.122352,0.332237,0.332386,...,0.776115,1.358406,3,0.029400,True,0.209885,0.776115,0.582291,True,0.443729
3,200,2443,400,77,0,https://www.baidu.com/,0.013203,0.094518,0.287696,0.287732,...,0.420138,0.420188,4,0.013203,True,0.193178,0.420138,0.000050,True,0.132406
4,200,82849,1147,79,0,https://www.wikipedia.org/,0.265244,0.348733,0.836824,0.836990,...,0.927058,1.124670,5,0.265244,True,0.488091,0.927058,0.197612,True,0.090068


# CONFIRM timing data is correct

In [110]:
from calculate_timings import calculate_sizes, calculate_timings 

In [116]:
df_data = pd.read_json("output/curl-timing-data-reorder-count100-sites500.json")
cols = ['rank','response_code', 'time_redirect', 'time_namelookup', 'time_connect',
       'time_appconnect', 'time_pretransfer','time_starttransfer', 'time_total']
df_success = df_data[df_data['response_code']==200][cols]

# Logic check

In [149]:
print(len(df_success))

37470


### t_tcp = time_connect - time_namelookup

In [141]:
# check for t_tcp = time_connect - time_namelookup
df_success[ df_success['time_connect']<df_success['time_namelookup'] ]

,rank,response_code,time_redirect,time_namelookup,time_connect,time_appconnect,time_pretransfer,time_starttransfer,time_total


### t_wait = time_starttransfer - time_pretransfer

In [142]:
# check for t_wait = time_starttransfer - time_pretransfer
df_success[ df_success['time_starttransfer']<df_success['time_pretransfer'] ]

,rank,response_code,time_redirect,time_namelookup,time_connect,time_appconnect,time_pretransfer,time_starttransfer,time_total


### t_dns = time_namelookup

In [148]:
# check for t_dns = time_namelookup (not time_namelookup - time_redirect if curl -L flag is ON)
print(len(df_success[ df_success['time_namelookup']<df_success['time_redirect'] ]))
print(len(df_success[ df_success['time_namelookup']>=df_success['time_redirect'] ]))

12412
25058


In [144]:
df_no_redirect = df_success[ df_success['time_namelookup']>=df_success['time_redirect'] ]
len( df_no_redirect[ df_no_redirect['time_redirect']>0 ] )

125

In [145]:
df_success[ df_success['time_namelookup']<=0 ]

,rank,response_code,time_redirect,time_namelookup,time_connect,time_appconnect,time_pretransfer,time_starttransfer,time_total


### t_rx = time_total - time_starttransfer

In [147]:
df_success[df_success['time_total']<df_success['time_starttransfer']]

,rank,response_code,time_redirect,time_namelookup,time_connect,time_appconnect,time_pretransfer,time_starttransfer,time_total


### t_calc = time_pretransfer - time_connect (not time_appconnect?)

In [156]:
len(df_success[df_success['time_pretransfer']<df_success['time_connect']])

0

In [153]:
len(df_success[df_success['time_pretransfer']<df_success['time_appconnect']])

936

In [159]:
len(df_success[df_success['time_starttransfer']<df_success['time_appconnect']])

693

### t_fbyte = time_starttransfer

In [212]:
df_success[ df_success['time_starttransfer']<=0 ]

,rank,response_code,time_redirect,time_namelookup,time_connect,time_appconnect,time_pretransfer,time_starttransfer,time_total


### t_ssl = time_appconnect - time_connect

In [192]:
df_weird = df_success[ df_success['time_appconnect']<df_success['time_connect'] ] 
df_weird.groupby('rank')['response_code'].count()

rank
280     1
319     1
339    87
407     1
Name: response_code, dtype: int64

In [189]:
df_sites[ df_sites['rank'].isin( df_weird['rank'].unique())]

,rank,site,IP,ASN
279,280,momoshop.com.tw,203.66.53.28,3462
318,319,ladbible.com,104.16.1.248,13335
338,339,cambridge.org,216.87.148.114,26415
406,407,fiverr.com,104.16.52.215,13335


### Based on https://blog.kenweiner.com/2014/11/http-request-timings-with-curl.html

- time_total > time_starttransfer > time_pretransfer > time_appconnect > time_connect > time_namelookup
- DNS Lookup = DNS Lookup (time_namelookup) - Start (0 ms) 
- TCP Connection = Pre Transfer (time_pretransfer) - DNS Lookup (time_namelookup)
    - time_connect <= time_pretransfer for time_appconnect = 0
- Content Generation = Start Transfer (time_starttransfer) - Pre Transfer (time_pretransfer) 
- Content Transfer = Total (time_total) - Start Transfer (time_starttransfer) 
- Time to first byte (TTFB) = time_starttransfer - time_appconnect (round trip time)
- Server calculation/response time = TTFB - (time_connect - time_namelookup)

## ONLY time_appconnect is malfuctioning

In [162]:
df_success.columns

Index(['rank', 'response_code', 'time_redirect', 'time_namelookup',
       'time_connect', 'time_appconnect', 'time_pretransfer',
       'time_starttransfer', 'time_total'],
      dtype='object')

In [199]:
for col_check in['time_namelookup', 'time_connect', 'time_appconnect', 
                 'time_pretransfer', 'time_starttransfer', 'time_total']:
    for col_name in ['time_namelookup', 'time_connect', 'time_appconnect', 
                     'time_pretransfer', 'time_starttransfer', 'time_total']:
        lengreater = len(  df_success[ df_success[col_check]<df_success[col_name] ]  )
        print ("%s < %s : %s " % (col_check, col_name, lengreater))

time_namelookup < time_namelookup : 0 
time_namelookup < time_connect : 37300 
time_namelookup < time_appconnect : 37349 
time_namelookup < time_pretransfer : 37470 
time_namelookup < time_starttransfer : 37470 
time_namelookup < time_total : 37470 
time_connect < time_namelookup : 0 
time_connect < time_connect : 0 
time_connect < time_appconnect : 33433 
time_connect < time_pretransfer : 37470 
time_connect < time_starttransfer : 37470 
time_connect < time_total : 37470 
time_appconnect < time_namelookup : 3 
time_appconnect < time_connect : 90 
time_appconnect < time_appconnect : 0 
time_appconnect < time_pretransfer : 36534 
time_appconnect < time_starttransfer : 36777 
time_appconnect < time_total : 37470 
time_pretransfer < time_namelookup : 0 
time_pretransfer < time_connect : 0 
time_pretransfer < time_appconnect : 936 
time_pretransfer < time_pretransfer : 0 
time_pretransfer < time_starttransfer : 37470 
time_pretransfer < time_total : 37470 
time_starttransfer < time_nameloo

In [201]:
df_weird2 = df_success[ df_success['time_pretransfer'] < df_success['time_appconnect'] ]


In [203]:
df_weird2.groupby('rank')['response_code'].count()

rank
86     98
128    98
257    83
280    97
315    99
319    98
339    12
364    98
407    98
415    99
482    56
Name: response_code, dtype: int64

In [204]:
df_sites[ df_sites['rank'].isin( df_weird2['rank'].unique())]

,rank,site,IP,ASN
85,86,wikia.com,151.101.64.194,54113
127,128,babytree.com,59.110.149.245,37963
256,257,ci123.com,202.102.90.109,23650
279,280,momoshop.com.tw,203.66.53.28,3462
314,315,namnak.com,185.49.84.250,43754
318,319,ladbible.com,104.16.1.248,13335
338,339,cambridge.org,216.87.148.114,26415
363,364,zol.com.cn,123.103.57.17,4808
406,407,fiverr.com,104.16.52.215,13335
414,415,chinadaily.com.cn,111.13.49.147,9808


In [205]:
df_asn_cdn = pd.read_pickle("output/df_asn_cdn.pkl")

In [206]:
df_asn_cdn[ df_asn_cdn['rank'].isin( df_weird['rank'].unique())]

,rank,site,IP,ASN,cdn_parsed,cdn_whois,cdn
279,280,momoshop.com.tw,203.66.53.28,3462,False,HiNet,HiNet
318,319,ladbible.com,104.16.1.248,13335,beta.ems.ladbiblegroup.com,Cloudflare,Cloudflare
338,339,cambridge.org,216.87.148.114,26415,False,False,False
406,407,fiverr.com,104.16.52.215,13335,False,Cloudflare,Cloudflare


In [207]:
df_asn_cdn[ df_asn_cdn['rank'].isin( df_weird2['rank'].unique())]

,rank,site,IP,ASN,cdn_parsed,cdn_whois,cdn
85,86,wikia.com,151.101.64.194,54113,Fastly,Fastly,Fastly
127,128,babytree.com,59.110.149.245,37963,False,False,False
256,257,ci123.com,202.102.90.109,23650,False,False,False
279,280,momoshop.com.tw,203.66.53.28,3462,False,HiNet,HiNet
314,315,namnak.com,185.49.84.250,43754,False,False,False
318,319,ladbible.com,104.16.1.248,13335,beta.ems.ladbiblegroup.com,Cloudflare,Cloudflare
338,339,cambridge.org,216.87.148.114,26415,False,False,False
363,364,zol.com.cn,123.103.57.17,4808,False,ChinaNetCenter,ChinaNetCenter
406,407,fiverr.com,104.16.52.215,13335,False,Cloudflare,Cloudflare
414,415,chinadaily.com.cn,111.13.49.147,9808,False,False,False


In [209]:
df_weird3 = df_success[ df_success['time_appconnect'] == df_success['time_connect'] ]
len(df_weird3)

3947

In [210]:
df_asn_cdn[ df_asn_cdn['rank'].isin( df_weird3['rank'].unique())]

,rank,site,IP,ASN,cdn_parsed,cdn_whois,cdn
19,20,weibo.com,114.134.80.162,9304,False,False,False
23,24,blogspot.com,172.217.161.9,15169,Google,Google,Google
26,27,netflix.com,54.76.161.146,16509,Open-Connect (Netflix),Amazon Technologies Inc.,Open-Connect (Netflix)
55,56,msn.com,13.82.28.61,8075,Akamai,Microsoft,Akamai
59,60,paypal.com,64.4.250.36,17012,False,False,False
81,82,thestartmagazine.com,184.168.131.241,26496,False,False,False
91,92,zhihu.com,118.89.204.109,45090,False,False,False
109,110,spotify.com,104.154.127.47,15169,scdn.co,Google,Google
138,139,theguardian.com,151.101.193.111,54113,i.guim.co.uk,Fastly,Fastly
140,141,avito.ru,185.89.12.132,201012,www.avito.st,False,www.avito.st


### Analyze bad sites - TODO CDF

In [24]:
#df_problem = CURLdata[CURLdata['response_code']!=200].groupby(['rank', 'response_code'])['url_effective'].count()
df_temp = df.groupby(['rank', 'pass'])['response_code'].count().reset_index()

In [29]:
df_pivot = df_temp.pivot(index='rank', columns='pass', values='response_code')

In [69]:
print("Number of sites with less than 33% response rate = ", len(df_pivot[df_pivot[True]<30]))
# 2 sites very low, 2 sites between 24-27
print("Number of sites with 33-34% response rate = ", len(df_pivot[ (df_pivot[True]>=33) & (df_pivot[True]<=34) ]))
# between 33-43% only 33-34% sites. next site with 44% response rate.
print("Number of sites with at least 70% response rate = ", len(df_pivot[df_pivot[True]>70]))

Number of sites with less than 33% response rate =  4
Number of sites with 33-34% response rate =  57
Number of sites with at least 70% response rate =  360


In [62]:
df_pivot[df_pivot[True]<33]

pass,False,True
rank,,
10,95.0,5.0
76,73.0,27.0
78,76.0,24.0
429,98.0,2.0


In [7]:
df_response = df_temp.pivot(index='rank', columns='response_code', values='url_effective').fillna(0)
df_response = df_response.join(df_sites.set_index('rank'))

df_response['total_pkt'] = df_response.sum(axis=1)
df_response['pass_rate'] = df_response[200]

df_response.head()

,0,200,204,301,302,403,404,429,500,502,503,520,site,IP,ASN,total_pkt,pass_rate
rank,,,,,,,,,,,,,,,,,
1,1.0,33.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,66.0,0.0,google.com,172.217.160.238,15169,100.0,33.0
2,1.0,99.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,youtube.com,172.217.31.14,15169,100.0,99.0
3,1.0,99.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,facebook.com,157.240.25.35,32934,100.0,99.0
4,2.0,98.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,baidu.com,123.125.115.110,4808,100.0,98.0
5,1.0,99.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,wikipedia.org,103.102.166.224,14907,100.0,99.0


### Sites with low response
- 76 of 500 sites could not be requested using curl at all (response rate = 0)
    - ISP block for porn and known adware sites which causes a timeout or 404.
    - Service sites that block at the server with port 80 and 443 closed. For ex: googleusercontent.com can't be reached directly using a HTTP or HTTPS request
    - Server side block for scripts with repeated requests. Although we tried to add delay and different user agent strings, certain servers can still detect that requests are not coming from a standard browser and either block us or redirect us.
- 64 of 500 sites had some valid (200) responses, but more than 70% were blocked.
    - first few requests may be allowed, but once the server/CDN realized that its a script it blocked our IP.

In [24]:
print("Total", len(df), "requests using curl")
print("Number of sites with no valid response at all:", len(df_response[df_response['pass_rate']==0]))
print("Number of sites with less than 70% response:", len(df_response[df_response['pass_rate']< 0.7 * df_response['total_pkt']]))


Total 10000 requests using curl
Number of sites with no valid response at all: 76
Number of sites with less than 70% response: 82


In [193]:
#df_response[df_response['pass_rate']==0]

# Reanalyze bad ssl and unreachable sites

In [91]:
df_bad_ssl = pd.read_pickle("output/df_timing_bad_sites_count20_sites50.pkl")
df_unreachable = pd.read_pickle("output/df_timing_unreachable_sites_count20-sites50.pkl")

In [100]:
print(len(df_unreachable))
df_unreachable[ df_unreachable['response_code']!=0 ]

620


,response_code,size_request,size_upload,size_download,size_header,time_redirect,time_namelookup,time_connect,time_appconnect,time_pretransfer,...,rank,timestamp,size_up,size_dw,t_dns,t_ssl,ssl_flag,t_fbyte,t_wait,t_rx


In [101]:
print(len(df_bad_ssl))

900


In [106]:
df_check = df_bad_ssl[ df_bad_ssl['response_code']!=0 ]
print(len(df_check))

879


In [194]:
#df_check[ df_check['ssl_flag']==True ]

### Testing pycurl

In [179]:
import pycurl, certifi

def retrieve_url(url):
    """
    Retrieves the requested url and records the response time.

    :param url: the url to retrieve
    :type url: string
    :return: A tuple of response times per exchange type if the request was successful, False otherwise
    :rtype: tuple/boolean
    """
    rvalue = False
    
    resp = defaultdict(int)
    
    try:
        curl = pycurl.Curl()
        curl.setopt(pycurl.CAINFO, certifi.where())
        # curl.setopt(pycurl.CAINFO, "C:/curl/curl-ca-bundle.crt") <-- same performance
        curl.setopt(pycurl.URL, url)  # set url
        curl.setopt(pycurl.WRITEFUNCTION, lambda x: None)
        curl.setopt(pycurl.SSL_VERIFYPEER, 1)
        curl.setopt(pycurl.SSL_VERIFYHOST, 2)
        curl.setopt(pycurl.FOLLOWLOCATION, 1)
        curl.perform()
        # Time from start until name resolving completed
        # https://curl.haxx.se/libcurl/c/CURLINFO_NAMELOOKUP_TIME.html
        resp['time_namelookup'] = curl.getinfo(pycurl.NAMELOOKUP_TIME)
        # Time from start until remote host or proxy completed.
        # https://curl.haxx.se/libcurl/c/CURLINFO_CONNECT_TIME.html
        resp['time_connect'] = curl.getinfo(pycurl.CONNECT_TIME)
        # Time from start until SSL/SSH handshake completed
        # https://curl.haxx.se/libcurl/c/CURLINFO_APPCONNECT_TIME.html
        resp['time_appconnect'] = curl.getinfo(pycurl.APPCONNECT_TIME)
        # Time from start until just when the first byte is received.
        # https://curl.haxx.se/libcurl/c/CURLINFO_STARTTRANSFER_TIME.html
        resp['time_pretransfer'] = curl.getinfo(pycurl.PRETRANSFER_TIME)
        resp['time_starttransfer'] = curl.getinfo(pycurl.STARTTRANSFER_TIME)
        resp['time_total'] = curl.getinfo(pycurl.TOTAL_TIME)
        resp['response_code'] = curl.getinfo(pycurl.RESPONSE_CODE)
        curl.close()
        rvalue = resp
    except Exception as e:
        logging.error(str(e))
        
    return rvalue


In [198]:
urls = ['https://www.momoshop.com.tw',
'https://www.ladbible.com',
'https://www.cambridge.org',
'https://www.fiverr.com',
'https://www.yahoo.com']

for url in urls:
    d=retrieve_url(url)
    print(url,d)

ERROR:root:(56, 'LibreSSL SSL_read: SSL_ERROR_SYSCALL, errno 54')


https://www.momoshop.com.tw False
https://www.ladbible.com defaultdict(<class 'int'>, {'time_namelookup': 0.067644, 'time_connect': 0.098076, 'time_appconnect': 0.204156, 'time_pretransfer': 0.20422, 'time_starttransfer': 0.244823, 'time_total': 0.246573, 'response_code': 403})
https://www.cambridge.org defaultdict(<class 'int'>, {'time_namelookup': 0.519602, 'time_connect': 0.5471980000000001, 'time_appconnect': 0.6386970000000001, 'time_pretransfer': 0.638757, 'time_starttransfer': 0.6712659999999999, 'time_total': 0.690382, 'response_code': 403})
https://www.fiverr.com defaultdict(<class 'int'>, {'time_namelookup': 0.068458, 'time_connect': 0.09799, 'time_appconnect': 0.130477, 'time_pretransfer': 0.098036, 'time_starttransfer': 0.575143, 'time_total': 0.974059, 'response_code': 200})
https://www.yahoo.com defaultdict(<class 'int'>, {'time_namelookup': 0.063345, 'time_connect': 0.149677, 'time_appconnect': 0.36377800000000005, 'time_pretransfer': 0.36386399999999997, 'time_starttran

In [181]:
print(retrieve_url("https://www.netflix.com"))

defaultdict(<class 'int'>, {'time_namelookup': 3.7e-05, 'time_connect': 4.1e-05, 'time_appconnect': 4.1e-05, 'time_pretransfer': 9.9e-05, 'time_starttransfer': 0.475275, 'time_total': 1.7892730000000001, 'response_code': 200})


## Testing

In [47]:
site = 'wikipedia.org'
url = 'https://www.'+site+'/'

In [71]:
#import sys
import json
#import re

# useful when threading
#sys.stdout.write(".")
#sys.stdout.flush()

p = subprocess.Popen(['curl', '-w', '@curl_time_format.txt', '-o', '/dev/null', '-s', url], stdout=subprocess.PIPE)
out, err = p.communicate()

# convert output string to python dict using json
json.loads(out.decode('UTF-8'))

#out = re.sub(r'(\d+),(\d+)', r'\1.\2', out.decode('UTF-8'))

{'url_effective': 'https://www.wikipedia.org/',
 'response_code': 200,
 'time_namelookup': 0.004959,
 'time_connect': 0.163835,
 'time_appconnect': 0.368026,
 'time_pretransfer': 0.36818,
 'time_redirect': 0.0,
 'time_starttransfer': 0.951108,
 'time_total': 1.299881}

In [155]:
import subprocess
import json
import time
import multiprocessing


list_of_websites = 'top-1m-new.csv' # location of alexa top websites as RANK,SITE\n
count = 10     # average timings over count loops of curl requests
nthreads = 2 # number of parallel threads for same url
# for each website COUNT time queries (in parallel)?
# for each result.response_code == '200' -> add to dict and find avg timings per website


def fetch_url(url):
    '''fetch url using curl
    connection timeout of 3.0 seconds
    result is [ dict {url_effective, response_code, time_namelookup, time_connect, time_appconnect,
    time_pretransfer, time_redirect, time_starttransfer, time_total}, error, exception ]
    '''
    try:
        p = subprocess.Popen(['curl', '--connect-timeout', '3.0', '-o', '/dev/null',  '-w', '@curl_time_format.txt',
                              '-s', url], stdout=subprocess.PIPE)
        out, err = p.communicate()
        result = json.loads(out.decode('UTF-8'))
        print("%r (%r) fetched with response code %r in %ss"
              % (url, result['url_effective'], result['response_code'], result['time_total']))    
    except Exception as e:
        print("Error fetching %r: Exception %s"
              % (url, e))
        result = None
    return result


In [156]:
def load_url_list(websites, nwebsites=500):
    """get top 500 of alexa websites csv <RANK, SITE> and append with 'https://www.' for curl request"""
    urls = defaultdict(int)
    from itertools import islice
    with open(websites) as f:
        for line in islice(f, nwebsites):
            rank, site = line.strip().split(',')
            url = 'https://www.' + site + '/'
            urls[rank] = url
    return urls

In [157]:
def main1():

    list_of_websites = 'top-1m-new.csv'  # location of alexa top websites as RANK,SITE\n
    count = 3  # average timings over count loops of curl requests
    nthreads = 3 # number of parallel threads for same url

    data = defaultdict(list)    # save result as json and load in pandas for averaging and analysis
    urls = load_url_list(list_of_websites, 5)
    url_counter = 0

    for rank, url in urls.items():

        url_counter += 1
        urls_parallel = [url for i in range(count)]
        print("Start time: %s, URL: %r, Rank: %s" % (time.time(), url, rank))

        pool = multiprocessing.Pool(processes=nthreads)
        pool_outputs = pool.map(fetch_url, urls_parallel)  # pool_output is a list of results

        pool.close()
        pool.join()

        for res in pool_outputs:
            if res is not None:
                [data[key].append(res[key]) for key in res.keys()]
                data['rank'].append(rank)

    #with open('output/curl-timing-data-test.json', 'w') as outfile:
    #    json.dump(data, outfile)

    return data

In [ ]:
d = main1()
pd.DataFrame(d)